In [10]:
import numpy as np
import tensorflow as tf
import keras
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN,Dense

#Loading the Dataset

In [11]:
max_feat = 1000 #maximum features
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=max_feat)

##printing the shape of the data
print(f'Training data shape :{x_train.shape} , Training lables shape :{y_train.shape}')
print(f'Testing data shape :{x_test.shape} , Testing lables shape :{y_test.shape}')


Training data shape :(25000,) , Training lables shape :(25000,)
Testing data shape :(25000,) , Testing lables shape :(25000,)


Padding


In [12]:
x_train = sequence.pad_sequences(x_train,padding ='pre',maxlen = 500)
x_test = sequence.pad_sequences(x_test,padding ='pre',maxlen = 500)

In [13]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

# Train simple RNN

In [14]:
model = Sequential()
model.add(Embedding(max_feat,128,input_length= 500)) ##Embedding layer 
model.add(SimpleRNN(128,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          128000    
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 161025 (629.00 KB)
Trainable params: 161025 (629.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
model.compile('adam','binary_crossentropy',metrics = ['accuracy'])

# Creating an Instance of early stopping Callback

In [16]:
from keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights = True) ## Stop if no improvement for more than 5 Epochs

In [17]:
history = model.fit(x_train,y_train,epochs=10,batch_size=32,validation_split = 0.2,callbacks = [earlystop])

Epoch 1/10
625/625 [==============================] - 64s 100ms/step - loss: 358630304.0000 - accuracy: 0.5876 - val_loss: 0.8017 - val_accuracy: 0.5332
Epoch 2/10
625/625 [==============================] - 55s 88ms/step - loss: 695.6038 - accuracy: 0.5792 - val_loss: 0.6616 - val_accuracy: 0.5884
Epoch 3/10
625/625 [==============================] - 61s 98ms/step - loss: 0.6274 - accuracy: 0.6239 - val_loss: 0.6217 - val_accuracy: 0.6312
Epoch 4/10
625/625 [==============================] - 54s 87ms/step - loss: 0.5768 - accuracy: 0.6862 - val_loss: 0.5631 - val_accuracy: 0.7210
Epoch 5/10
625/625 [==============================] - 51s 81ms/step - loss: 0.5212 - accuracy: 0.7556 - val_loss: 0.5184 - val_accuracy: 0.7540
Epoch 6/10
625/625 [==============================] - 52s 83ms/step - loss: 0.4971 - accuracy: 0.7663 - val_loss: 0.5059 - val_accuracy: 0.7666
Epoch 7/10
625/625 [==============================] - 52s 84ms/step - loss: 0.4747 - accuracy: 0.7821 - val_loss: 0.4847 - va

Save the model

In [18]:
model.save('simple_rnn_imdb.h5')

e:\RNN_project\DL_simple_RNN_project\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
